In [1]:
!pip install nopecha


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import requests
from selenium import webdriver

NOPECHA_KEY = 'sub_1QY6J2CRwBwvt6ptOboZL3Yw'  # Replace with your key.

options = webdriver.chrome.options.Options()
options.add_argument('--no-sandbox')
options.add_argument('--disable-infobars')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_experimental_option('excludeSwitches', ['enable-automation'])
options.add_experimental_option('useAutomationExtension', False)

# Download the latest NopeCHA crx extension file.
# You can also supply a path to a directory with unpacked extension files.
with open('ext.crx', 'wb') as f:
    f.write(requests.get('https://nopecha.com/f/ext.crx').content)
options.add_extension('ext.crx')

# Start the driver.
driver = webdriver.Chrome(options=options)

# Set the subscription key for the extension by visiting this URL.
# You can programmatically import all extension settings using this method.
# To learn more, go to "Export Settings" in the extension popup.
driver.get(f"https://nopecha.com/setup#{NOPECHA_KEY}")

# Go to any page with a CAPTCHA and the extension will automatically solve it.
driver.get('https://www.cyberbackgroundchecks.com')

In [3]:
!pip install pyppeteer


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import asyncio
import json
from pyppeteer import launch
import requests
import nest_asyncio
import asyncio

nest_asyncio.apply()

# Replace with your 2Captcha API key
API_KEY = 'b2af1b59536c7f5a659f232ac62f1bf1'

async def main():
    # Launch browser
    browser = await launch(headless=False, args=[
        '--no-sandbox',
        '--disable-setuid-sandbox',
        '--disable-blink-features=AutomationControlled',
        '--disable-infobars',
    ])
    page = await browser.newPage()

    # Set JavaScript and headers
    await page.setJavaScriptEnabled(True)
    await page.setExtraHTTPHeaders({
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.9',
        'Connection': 'keep-alive',
        'DNT': '1',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'
    })

    # Load cookies from a previous session
    try:
        with open('cookies.json', 'r') as f:
            cookies = json.load(f)
            await page.setCookie(*cookies)
            print("Cookies loaded.")
    except FileNotFoundError:
        print("No previous cookies found.")

    # Navigate to the target page
    target_url = "https://www.cyberbackgroundchecks.com"  # Replace with the actual URL
    await page.goto(target_url, waitUntil='domcontentloaded')

    # Save cookies after visiting
    browser_cookies = await page.cookies()
    with open('cookies.json', 'w') as f:
        json.dump(browser_cookies, f)
        print("Cookies saved.")

    # Detect Turnstile CAPTCHA
    content = await page.content()
    if "cf-turnstile" in content:
        print("Turnstile CAPTCHA detected.")
        site_key = await page.evaluate('document.querySelector("[data-sitekey]").getAttribute("data-sitekey")')
        print(f"Site Key: {site_key}")
        solution = await solve_turnstile_captcha(site_key, target_url)
        if solution:
            await page.evaluate(f'''
                document.querySelector("[data-callback]").dispatchEvent(new Event("cf-challenge-resolve", {{
                    detail: "{solution}"
                }}));
            ''')
            print("CAPTCHA solved.")
        else:
            print("Failed to solve CAPTCHA.")
    else:
        print("No CAPTCHA detected.")

    # Keep browser open for debugging
    await asyncio.sleep(10)
    # await browser.close()

async def solve_turnstile_captcha(site_key, page_url):
    """Solve Turnstile CAPTCHA using 2Captcha."""
    payload = {
        'key': API_KEY,
        'method': 'turnstile',
        'sitekey': site_key,
        'pageurl': page_url,
        'json': 1
    }
    response = requests.post("https://2captcha.com/in.php", data=payload)
    if response.status_code == 200 and response.json().get('status') == 1:
        captcha_id = response.json().get('request')
        print(f"Captcha submitted, ID: {captcha_id}")
        return await poll_for_solution(captcha_id)
    else:
        print("Failed to submit CAPTCHA:", response.text)
        return None

async def poll_for_solution(captcha_id):
    """Poll 2Captcha for the solution."""
    url = f"https://2captcha.com/res.php?key={API_KEY}&action=get&id={captcha_id}&json=1"
    while True:
        response = requests.get(url)
        if response.status_code == 200:
            result = response.json()
            if result.get('status') == 1:
                return result.get('request')
            elif result.get('request') == 'CAPCHA_NOT_READY':
                print("CAPTCHA not ready, retrying...")
                await asyncio.sleep(5)
            else:
                print("Error solving CAPTCHA:", result.get('request'))
                return None
        else:
            print("Error polling for CAPTCHA solution:", response.text)
            return None

await main()


Cookies loaded.
Cookies saved.
No CAPTCHA detected.


In [ ]:
import asyncio
import json
import time
import requests
from pyppeteer import launch
import nest_asyncio
import asyncio

nest_asyncio.apply()

# Replace with your 2Captcha API key
API_KEY = 'b2af1b59536c7f5af232ac62f1bf1'

async def main():
    browser = await launch(headless=False, args=['--no-sandbox', '--disable-setuid-sandbox'])
    page = await browser.newPage()

    # Function to inject Turnstile interception script
    async def inject_turnstile_interceptor():
        await page.evaluateOnNewDocument('''
            () => {
                console.clear = () => console.log('Console was cleared');
                const i = setInterval(() => {
                    if (window.turnstile) {
                        clearInterval(i);
                        window.turnstile.render = (a, b) => {
                            let params = {
                                sitekey: b.sitekey,
                                pageurl: window.location.href,
                                data: b.cData,
                                pagedata: b.chlPageData,
                                action: b.action,
                                userAgent: navigator.userAgent,
                                json: 1
                            };
                            console.log('intercepted-params:' + JSON.stringify(params));
                            window.cfCallback = b.callback;
                            return;
                        };
                    }
                }, 50);
            }
        ''')

    # Inject the interceptor when the page is first loaded
    await inject_turnstile_interceptor()

    # Re-inject the interceptor on every page load
    page.on('load', lambda: asyncio.ensure_future(inject_turnstile_interceptor()))

    # Define a handler to process console messages
    page.on('console', lambda msg: asyncio.ensure_future(console_message_handler(msg, page)))

    # Navigate to the initial page
    await page.goto('https://www.cyberbackgroundchecks.com/people/michael-simpson/md/queenstown')
    await page.type('#SearchCriteriaViewModel_FirstName', 'Michael')
    await page.type('#SearchCriteriaViewModel_LastName', 'Simpson')
    # Wait for the button to appear
    await page.waitForSelector('#button-search-by-name')

    # Click the button and wait for the navigation
    navigation_promise = asyncio.create_task(page.waitForNavigation())
    await page.click('#button-search-by-name')
    # print('Button clicked. Waiting for navigation...')

    # Wait for navigation to complete
    await navigation_promise

    # Print the new URL
    new_url = page.url
    print('Navigated to:', new_url)

    # Inject Turnstile logic on the new page
    await inject_turnstile_interceptor()
    # print('Turnstile interception logic re-injected.')

    # Add further actions here, such as waiting for specific elements or interacting with the page
    if "people" in new_url:
        # print("Successfully navigated to a profile page.")

    # Wait to observe the behavior or perform further actions
    await asyncio.sleep(10)

async def console_message_handler(msg, page):
    txt = msg.text()
    if 'intercepted-params:' in txt:
        params = json.loads(txt.replace('intercepted-params:', ''))
        print('Intercepted parameters:', params)

        # Prepare payload for 2Captcha
        payload = {
            'key': API_KEY,
            'method': 'turnstile',
            'sitekey': params['sitekey'],
            'pageurl': params['pageurl'],
            'data': params['data'],
            'pagedata': params['pagedata'],
            'action': params['action'],
            'useragent': params['userAgent'],
            'json': 1,
        }

        # Send captcha to 2Captcha
        response = requests.post('https://2captcha.com/in.php', data=payload)
        if response.status_code == 200 and response.json().get('status') == 1:
            captcha_id = response.json().get('request')
            print('Captcha submitted, ID:', captcha_id)

            # Poll for the solution
            solution = await poll_for_solution(captcha_id)
            if solution:
                # Execute the callback with the solution
                await page.evaluate(f'cfCallback("{solution}");')
                print('Captcha solved and callback executed.')
        else:
            print('Failed to submit captcha:', response.text)

async def poll_for_solution(captcha_id):
    while True:
        time.sleep(5)
        response = requests.get(f'https://2captcha.com/res.php?key={API_KEY}&action=get&id={captcha_id}&json=1')
        if response.status_code == 200:
            result = response.json()
            if result.get('status') == 1:
                return result.get('request')
            elif result.get('request') == 'CAPCHA_NOT_READY':
                print('Captcha not ready, retrying...')
            else:
                print('Error:', result.get('request'))
                return None
        else:
            print('Error fetching captcha solution:', response.text)
            return None

# Run the main function
await main()

Task exception was never retrieved
future: <Task finished name='Task-31' coro=<console_message_handler() done, defined at C:\Users\Ali Hassan\AppData\Local\Temp\ipykernel_15608\4216868836.py:84> exception=TypeError("'str' object is not callable")>
Traceback (most recent call last):
  File "c:\Users\Ali Hassan\.conda\envs\aiscrape\lib\asyncio\tasks.py", line 280, in __step
    result = coro.send(None)
  File "C:\Users\Ali Hassan\AppData\Local\Temp\ipykernel_15608\4216868836.py", line 85, in console_message_handler
    txt = msg.text()
TypeError: 'str' object is not callable


In [3]:
import asyncio
import json
import time
import requests
import nest_asyncio
from pyppeteer import launch

nest_asyncio.apply()

API_KEY = 'b2af1b59536c7f5a659f232ac62f1bf1'

async def main():
    browser = await launch(headless=False, args=['--no-sandbox', '--disable-setuid-sandbox'])
    page = await browser.newPage()
    page.on('console', lambda msg: asyncio.ensure_future(console_message_handler(msg, page)))

    # Initial navigation
    await page.goto('https://www.cyberbackgroundchecks.com')
    await page.type('#SearchCriteriaViewModel_FirstName', 'Michael')
    await page.type('#SearchCriteriaViewModel_LastName', 'Simpson')
    await page.waitForSelector('#button-search-by-name')

    # Click button and wait for the next page (the challenge page)
    navigation_promise = asyncio.create_task(page.waitForNavigation())
    await page.click('#button-search-by-name')
    print('Button clicked. Waiting for navigation...')
    await navigation_promise

    new_url = page.url
    print('Navigated to:', new_url)

    # If this URL includes __cf_chl_rt_tk, it's a Cloudflare challenge page.
    # Wait for the Turnstile iframe to load.
    # On challenge pages, there's usually an iframe from challenges.cloudflare.com
    turnstile_frame = await wait_for_turnstile_frame(page)
    if turnstile_frame is not None:
        print("Turnstile iframe found:", turnstile_frame.url)
        await inject_turnstile_interceptor_in_frame(turnstile_frame)
    else:
        print("No turnstile iframe found. Cannot intercept.")

    # Wait to observe if interception occurs
    await asyncio.sleep(15)

async def wait_for_turnstile_frame(page, timeout=15000):
    # Wait up to timeout ms for an iframe from challenges.cloudflare.com
    start = time.time()
    while True:
        for frame in page.frames:
            # Cloudflare turnstile iframes often originate from challenges.cloudflare.com
            if "challenges.cloudflare.com" in frame.url:
                return frame
        if (time.time() - start) * 1000 > timeout:
            return None
        await asyncio.sleep(0.5)

async def inject_turnstile_interceptor_in_frame(frame):
    # Inject code inside the iframe that hooks into turnstile.render
    await frame.evaluate('''
        () => {
            console.log("[Iframe] Turnstile interceptor injection started.");
            const checkInterval = setInterval(() => {
                if (window.turnstile && typeof window.turnstile.render === 'function') {
                    clearInterval(checkInterval);
                    const originalRender = window.turnstile.render;
                    window.turnstile.render = (element, params) => {
                        const interceptedParams = {
                            sitekey: params.sitekey,
                            pageurl: window.location.href,
                            data: params.cData,
                            pagedata: params.chlPageData,
                            action: params.action,
                            userAgent: navigator.userAgent,
                            json: 1
                        };
                        console.log("intercepted-params:" + JSON.stringify(interceptedParams));
                        window.cfCallback = params.callback;
                        return originalRender.call(window.turnstile, element, params);
                    };
                    console.log("[Iframe] Turnstile render hooked successfully.");
                }
            }, 50);
        }
    ''')

async def console_message_handler(msg, page):
    txt = msg.text()
    if 'intercepted-params:' in txt:
        params = json.loads(txt.replace('intercepted-params:', ''))
        print('Intercepted parameters:', params)

        # Override pageurl with current URL, just in case
        params['pageurl'] = page.url

        payload = {
            'key': API_KEY,
            'method': 'turnstile',
            'sitekey': params['sitekey'],
            'pageurl': params['pageurl'],
            'data': params['data'],
            'pagedata': params['pagedata'],
            'action': params['action'],
            'useragent': params['userAgent'],
            'json': 1,
        }

        response = requests.post('https://2captcha.com/in.php', data=payload)
        if response.status_code == 200 and response.json().get('status') == 1:
            captcha_id = response.json().get('request')
            print('Captcha submitted, ID:', captcha_id)

            solution = await poll_for_solution(captcha_id)
            if solution:
                # Execute the callback to solve the captcha
                await page.evaluate(f'cfCallback("{solution}");')
                print('Captcha solved and callback executed.')
        else:
            print('Failed to submit captcha:', response.text)

async def poll_for_solution(captcha_id):
    while True:
        time.sleep(5)
        response = requests.get(f'https://2captcha.com/res.php?key={API_KEY}&action=get&id={captcha_id}&json=1')
        if response.status_code == 200:
            result = response.json()
            if result.get('status') == 1:
                return result.get('request')
            elif result.get('request') == 'CAPCHA_NOT_READY':
                print('Captcha not ready, retrying...')
            else:
                print('Error:', result.get('request'))
                return None
        else:
            print('Error fetching captcha solution:', response.text)
            return None

# Run the main function
await main()


Task exception was never retrieved
future: <Task finished name='Task-372' coro=<console_message_handler() done, defined at C:\Users\Ali Hassan\AppData\Local\Temp\ipykernel_23976\140403701.py:86> exception=TypeError("'str' object is not callable")>
Traceback (most recent call last):
  File "c:\Users\Ali Hassan\.conda\envs\aiscrape\lib\asyncio\tasks.py", line 280, in __step
    result = coro.send(None)
  File "C:\Users\Ali Hassan\AppData\Local\Temp\ipykernel_23976\140403701.py", line 87, in console_message_handler
    txt = msg.text()
TypeError: 'str' object is not callable
Task exception was never retrieved
future: <Task finished name='Task-375' coro=<console_message_handler() done, defined at C:\Users\Ali Hassan\AppData\Local\Temp\ipykernel_23976\140403701.py:86> exception=TypeError("'str' object is not callable")>
Traceback (most recent call last):
  File "c:\Users\Ali Hassan\.conda\envs\aiscrape\lib\asyncio\tasks.py", line 280, in __step
    result = coro.send(None)
  File "C:\Users\

Button clicked. Waiting for navigation...
Navigated to: https://www.cyberbackgroundchecks.com/people/michael-simpson
No turnstile iframe found. Cannot intercept.
